<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD001_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [63]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from scikeras.wrappers import KerasRegressor

In [29]:
sns.set_palette('colorblind')

In [30]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Preparation

In [5]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [6]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [7]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [8]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Validation Set

In [9]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(train):
    gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  
    for idx_train, idx_val in gss.split(train,groups=train["unit_number"]):
        # print('train_split_engines', train.iloc[idx_train]['unit_number'].unique(), '\n')
        # print('validate_split_engines', train.iloc[idx_val]['unit_number'].unique(), '\n')

        df_train = train.iloc[idx_train].copy()
        df_val = train.iloc[idx_val].copy()

    return df_train, df_val

In [10]:
df_train, df_val = train_val_split(train)

In [11]:
X_t_train, y_t_train = proc.X_y_train_divide(df_train)
X_t_val, y_t_val = proc.X_y_train_divide(df_val)

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [12]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [13]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [14]:
X_test = test_last

### Remaining Useful Life (RUL)

In [15]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [16]:
X_train, y_train = proc.X_y_train_divide(train)

In [17]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [18]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [19]:
eval = Evaluation()

In [20]:
search = HyperparameterSearch()

# MLP

## Linear RUL

In [77]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [73]:
def create_model(optim=Adam, layer_nodes=[16,32,64], 
                 dropout=0.1, activation="relu", learning_rate=1e-4, 
                 loss='mean_squared_error'):
    train_cols = sensors_cols
    input_dim = len(train_cols)

    model = Sequential()
    model.add(Dense(layer_nodes[0], input_dim=input_dim, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[2], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate))

    return model
reg = KerasRegressor(model=create_model)

In [74]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [75]:
model = Pipeline([
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model,verbose=0, callbacks=[es], validation_split=0.2))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))])

In [78]:
ED1 = ExponentialDecay(initial_learning_rate=1e-2, 
                       decay_steps=100000, decay_rate=0.96)
ED2 = ExponentialDecay(initial_learning_rate=1e-2, 
                       decay_steps=100000, decay_rate=0.8)
ED3 = ExponentialDecay(initial_learning_rate=1e-1, 
                       decay_steps=100000, decay_rate=0.96)

In [79]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4,ED1,ED2,ED3],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=3)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 3 folds for each of 14 candidates, totalling 42 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 3 folds for each of 5 candidates, totalling 15 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 3 folds for each of 2 candidates, totalling 6 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.5, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.01, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'tanh', 'mlp__epochs': 30}
Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf43e6e190>], epochs=30, model=<function create_model at 0x7fdf49de63b0>, model__activation='tanh', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__learning_rate=0.01, model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.5, verbose=0))])


Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.5, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.01, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'tanh', 
'mlp__epochs': 30}
```

In [80]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, verbose=0, callbacks=[es], 
                                validation_split=0.5, epochs=30, 
                                model__optim=SGD, model__layer_nodes=[128,256,512],
                                model__dropout=0.4, model__activation='tanh'))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], epochs=30, model=<function create_model at 0x7fdf49de63b0>, model__activation='tanh', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.5, verbose=0))])

In [81]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.629,RMSE=-41.928
(CV) R2=0.623,RMSE=-41.758


In [82]:
eval.show_result(y_test, model.predict(X_test))

R2=0.515,RMSE=-28.927


In [83]:
model = Pipeline([
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model,verbose=0, callbacks=[es], validation_split=0.2))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))])

In [90]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV


search = HalvingRandomSearchCV(model, param_distributions,
                               min_resources=500, scoring='r2', 
                               random_state=42, verbose=2, cv=3)


search.fit(X_train, y_train);
print("Best params: ", search.best_params_)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
[CV] END mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[32, 64, 128], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fdf439f5810>, mlp__model__optim=<class 'keras.optimizer_v2.adam.Adam'>, mlp__validation_split=0.2, scaler=StandardScaler(); total time=   1.4s
[CV] END mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[32, 64, 128], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fdf439f5810>, mlp__model__optim=<class 'keras.optimizer_v2.adam.Adam'>, mlp__validation_split=0.2, scaler=StandardScaler(); total time=   1.9s
[CV] END mlp__epo

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=0.001, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.5, scaler=StandardScaler(); total time=   1.6s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=0.001, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.5, scaler=StandardScaler(); total time=   1.1s


...

[CV] END mlp__epochs=5, mlp__model__activation=relu, mlp__model__dropout=0.4, mlp__model__layer_nodes=[64, 128, 256], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fdf439f58d0>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.1, scaler=StandardScaler(); total time=   3.7s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__epochs=5, mlp__model__activation=relu, mlp__model__dropout=0.4, mlp__model__layer_nodes=[64, 128, 256], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fdf439f58d0>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.1, scaler=StandardScaler(); total time=   3.5s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__epochs=5, mlp__model__activation=relu, mlp__model__dropout=0.4, mlp__model__layer_nodes=[64, 128, 256], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fdf439f58d0>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.1, scaler=StandardScaler(); total time=   3.5s
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.5, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.01, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'tanh', 'mlp__epochs': 30}


In [86]:
dir(search)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_input_parameters',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_checked_cv_orig',
 '_estimator_type',
 '_format_results',
 '_generate_candidate_params',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_n_samples_orig',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'aggressive_elimination',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',


In [89]:
search.score

<bound method BaseSearchCV.score of HalvingRandomSearchCV(cv=3,
                      estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('mlp',
                                                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))]),
                      min_resources=500,
                      param_distributions={'mlp__epochs': [1, 5, 10, 20, 30,
                                                           50],
                                           'mlp__model__activation': ['relu'...
                                           'mlp__model__optim': [<class 'keras.optimizer_v2.adam.Adam'>,
                                                                 <class 'keras.optimizer_v2.gradient_descent.SGD'>,
                                                                 <class 'keras.optimizer_v2.rmsprop.R

## Linear RUL

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 20, 'tree_reg__min_impurity_decrease': 3.5, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 200, 'tree_reg__criterion': 'friedman_mse', 'tree_reg__ccp_alpha': 0.675}
Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                          

Best Model:
```
('tree_reg',
DecisionTreeRegressor(ccp_alpha=0.675,
                    criterion='friedman_mse', max_depth=200,
                    max_features='log2',
                    min_impurity_decrease=3.5,
                    min_samples_leaf=20, random_state=42))
```

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=0.675,
                                    criterion='friedman_mse', max_depth=200,
                                    max_features='log2',
                                    min_impurity_decrease=3.5,
                                    min_samples_leaf=20, random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                                       criterion='friedman_mse', max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=3.5,
                                       min_samples_leaf=20, random_state=42))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.645,RMSE=-41.037
(CV) R2=0.537,RMSE=-46.205


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.298,RMSE=-34.821


Worse than simple linear regression

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}, 'trf_reg__regressor__splitter': 'best', 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 5.5, 'trf_reg__regressor__max_features': 'sqrt', 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'friedman_mse', 'trf_reg__r

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}}

DecisionTreeRegressor(ccp_alpha=0.375,
                    criterion='friedman_mse',
                    max_depth=200,
                    max_features='sqrt',
                    min_impurity_decrease=5.5,
                    min_samples_leaf=10,
                    random_state=42),
```



In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.375,
                            criterion='friedman_mse',
                            max_depth=200,
                            max_features='sqrt',
                            min_impurity_decrease=5.5,
                            min_samples_leaf=10,
                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':98})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.375,
                                                                            criterion='friedman_mse',
                                                                            max_depth=200,
                                                                            max_features='sqrt',
                                                                            min_impurity_decrease=5.5,
                                                                            min_samples_leaf=10,
                                                                            random_state=42),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                                            kw_args={'a_max': 98,
                

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.815,RMSE=-13.536
(CV) R2=0.781,RMSE=-14.680


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.819,RMSE=-14.277


The best result we had so far in training and in testing

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": [None, "sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'random', 'tree_reg__min_samples_split': 15, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 6.9, 'tree_reg__max_features': 'sqrt', 'tree_reg__max_depth': 100, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 1.025, 'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degree': 3}
Pipeline(steps=[('

Best Model:
```
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                    max_features='sqrt',
                    min_impurity_decrease=6.9,
                    min_samples_leaf=15,
                    min_samples_split=15, random_state=42,
                    splitter='random'))
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                        max_features='sqrt',
                                        min_impurity_decrease=6.9,
                                        min_samples_leaf=15,
                                        min_samples_split=15, random_state=42,
                                        splitter='random'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                       max_features='sqrt',
                                       min_impurity_decrease=6.9,
                                       min_samples_leaf=15,
                                       min_samples_split=15, random_state=42,
                                       splitter='random'))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.656
(CV) R2=0.550,RMSE=-45.641


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.311,RMSE=-34.484


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": [None, "sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}, 'trf_reg__regressor__splitter': 'random', 'trf_reg__regressor__min_samples_split': 20, 'trf_reg__regressor__min_samples_leaf': 2, 'trf_reg__regressor__min_impurity_decrease': 8.5, 'trf_reg__regressor__max_features': None, 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'squared_error', 'trf_reg__

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.85,
                        max_depth=200,
                        min_impurity_decrease=8.5,
                        min_samples_leaf=2,
                        min_samples_split=20,
                        random_state=42,
                        splitter='random'),
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=1.85,
                                            max_depth=200,
                                            min_impurity_decrease=8.5,
                                            min_samples_leaf=2,
                                            min_samples_split=20,
                                            random_state=42,
                                            splitter='random'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':71})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=1.85,
                                                                            max_depth=200,
                                                                            min_impurity_decrease=8.5,
                                                                            min_samples_leaf=2,
                                                                            min_samples_split=20,
                                                                            random_state=42,
                                                                            splitter='random'),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                               

In [ ]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.772,RMSE=-9.983
(CV) R2=0.772,RMSE=-9.954


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.711,RMSE=-12.322


Using the polynominal features actually worsened the perfomance a little bit, when comparing with not using them and the non-linear RUL.